<a href="https://colab.research.google.com/github/Edward1304/Signals-Theory-Edward/blob/main/CNN_Prediction_and_detection_(labeled_signals).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Red Neuronal Convolucional para  prediccion y deteccion de Señales


# Edward Fabian Goyeneche Velandia.

# Teoria de Señales